In [2]:
import time
import torch
from torch.autograd import Variable
import torch.nn.functional as F
from wavenet_model import *
from audio_data import WavenetDataset
from wavenet_training import *
from model_logging import *
from scipy.io import wavfile

dtype = torch.FloatTensor
ltype = torch.LongTensor

use_cuda = torch.cuda.is_available()
if use_cuda:
    print('use gpu')
    dtype = torch.cuda.FloatTensor
    ltype = torch.cuda.LongTensor
else: 
    print("no gpu found")

no gpu found


In [3]:
model = WaveNetModel(layers=10,
                     blocks=4,
                     dilation_channels=32,
                     residual_channels=32,
                     skip_channels=32,
                     output_length=64,
                     dtype=dtype)
#model = load_latest_model_from('snapshots', use_cuda=use_cuda)
if use_cuda:
    model.cuda()

print('model: ', model)
print('receptive field: ', model.receptive_field)
print('parameter count: ', model.parameter_count())

model:  WaveNetModel(
  (filter_convs): ModuleList(
    (0): Conv1d (32, 32, kernel_size=(2,), stride=(1,), bias=False)
    (1): Conv1d (32, 32, kernel_size=(2,), stride=(1,), bias=False)
    (2): Conv1d (32, 32, kernel_size=(2,), stride=(1,), bias=False)
    (3): Conv1d (32, 32, kernel_size=(2,), stride=(1,), bias=False)
    (4): Conv1d (32, 32, kernel_size=(2,), stride=(1,), bias=False)
    (5): Conv1d (32, 32, kernel_size=(2,), stride=(1,), bias=False)
    (6): Conv1d (32, 32, kernel_size=(2,), stride=(1,), bias=False)
    (7): Conv1d (32, 32, kernel_size=(2,), stride=(1,), bias=False)
    (8): Conv1d (32, 32, kernel_size=(2,), stride=(1,), bias=False)
    (9): Conv1d (32, 32, kernel_size=(2,), stride=(1,), bias=False)
    (10): Conv1d (32, 32, kernel_size=(2,), stride=(1,), bias=False)
    (11): Conv1d (32, 32, kernel_size=(2,), stride=(1,), bias=False)
    (12): Conv1d (32, 32, kernel_size=(2,), stride=(1,), bias=False)
    (13): Conv1d (32, 32, kernel_size=(2,), stride=(1,), bias

In [7]:
batch_size = 32
input_data = Variable(torch.zeros([batch_size, 1, model.receptive_field + model.output_length - 1]))
print(input_data)

Variable containing:
( 0  ,.,.) = 
   0   0   0  ...    0   0   0

( 1  ,.,.) = 
   0   0   0  ...    0   0   0

( 2  ,.,.) = 
   0   0   0  ...    0   0   0
 ... 

( 29 ,.,.) = 
   0   0   0  ...    0   0   0

( 30 ,.,.) = 
   0   0   0  ...    0   0   0

( 31 ,.,.) = 
   0   0   0  ...    0   0   0
[torch.FloatTensor of size 32x1x4156]



In [13]:
with torch.autograd.profiler.profile(enabled=True, use_cuda=True) as prof:
    out = model(input_data)
    loss = F.cross_entropy(out.squeeze(), Variable(torch.zeros([batch_size * model.output_length]).type(ltype)))
    loss.backward()
print(prof.key_averages().table(sort_by='cpu_time_total'))

TypeError: __init__() got an unexpected keyword argument 'use_cuda'

In [ ]:
prof.export_chrome_trace('profiling/latest_trace.json')

In [11]:
with torch.autograd.profiler.profile() as prof:
    model.generate_fast(num_samples=100)
print(prof.key_averages().table(sort_by='cpu_time_total'))

one generating step does take approximately 0.012284069061279297 seconds)
---------------  ---------------  ---------------  ---------------  ---------------  ---------------
Name                    CPU time        CUDA time            Calls        CPU total       CUDA total
---------------  ---------------  ---------------  ---------------  ---------------  ---------------
view                     6.082us          0.000us                1          6.082us          0.000us
squeeze                  1.857us          0.000us              100        185.680us          0.000us
threshold                3.185us          0.000us              100        318.454us          0.000us
div                      3.349us          0.000us              100        334.918us          0.000us
softmax                  6.903us          0.000us              100        690.307us          0.000us
unsqueeze                1.704us          0.000us             4000       6816.902us          0.000us
mul              